### TASK: To Create a Python program that fetches real-time weather data (temperature, humidity) from a public weather API (e.g., OpenWeatherMap), logs it at regular intervals for multiple locations, stores the data in an SQLite database, and analyzes it by calculating daily and weekly averages for each location.

In [21]:
import pandas as pd
import requests
import sqlite3
import schedule
import time
import os
import logging
from datetime import datetime, timezone

locations = pd.read_csv("location_list.csv", header=None)
locations.head()

,0
0,New York
1,London
2,Tokyo
3,Berlin
4,Paris


In [22]:
locations.columns = ["City"]
locations.head()

,City
0,New York
1,London
2,Tokyo
3,Berlin
4,Paris


In [23]:
cities = locations["City"].tolist() # city list to calculate weather
print(cities)

['New York', 'London', 'Tokyo', 'Berlin', 'Paris', 'Sydney', 'Los Angeles', 'Madrid', 'Rome', 'Moscow', 'Cairo', 'Dubai', 'Singapore', 'Seoul', 'Toronto', 'Mumbai']


In [24]:
API_KEY =  os.getenv("API_KEY") 
conn = sqlite3.connect("weather_data.db")
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS weather_log (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        city TEXT,
        temperature REAL,
        humidity REAL,
        timestamp DATETIME
    )
""")
conn.commit()
conn.close()

In [25]:
logging.basicConfig(
    filename="logs.txt",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

In [ ]:
def fetch_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()
        if response.status_code == 200:
            return {
                "city": city,
                "temperature": data["main"]["temp"],
                "humidity": data["main"]["humidity"],
                "timestamp": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")
            }
        else:
            logging.warning(f"API error for {city}: {data.get('message')}")
            return None
    except Exception as e:
        logging.error(f"Error fetching weather for {city}: {e}")
        return None

def insert_weather(data):
    if data:
        print(data)
        conn = sqlite3.connect("weather_data.db")
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO weather_log (city, temperature, humidity, timestamp)
            VALUES (?, ?, ?, ?)
        """, (data["city"], data["temperature"], data["humidity"], data["timestamp"]))
        conn.commit()
        conn.close()
        logging.info(f"Inserted weather data for {data['city']}")

def calculate_aggregates():
    try:
        conn = sqlite3.connect("weather_data.db")
        df = pd.read_sql_query("SELECT * FROM weather_log", conn)
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df["date"] = df["timestamp"].dt.date
        df["week"] = df["timestamp"].dt.isocalendar().week # to calc weekly average 

        daily_avg = df.groupby(["city", "date"])[["temperature", "humidity"]].mean().round(2)
        weekly_avg = df.groupby(["city", "week"])[["temperature", "humidity"]].mean().round(2)

        daily_avg.to_csv("daily_average_cal.csv")
        weekly_avg.to_csv("weekly_average_cal.csv")

        logging.info("Saved daily and weekly averages to CSV files.")
        conn.close()
    except Exception as e:
        logging.error(f"Error calculating aggregates: {e}")

In [27]:
def log_weather_cities():
    logging.info("Logging started")
    for city in cities:
        data = fetch_weather(city)
        insert_weather(data)
    logging.info("Completed weather logging for all cities")

In [28]:
schedule.every(1).minutes.do(log_weather_cities) # scheduled for 1 min - next does the loging
while True:
    schedule.run_pending()
    time.sleep(5)  # to check every 5 secs if there is a pending schedule or not 

{'city': 'New York', 'temperature': 7.63, 'humidity': 93, 'timestamp': '2025-03-20 15:28:28'}
{'city': 'London', 'temperature': 19.36, 'humidity': 51, 'timestamp': '2025-03-20 15:28:28'}
{'city': 'Tokyo', 'temperature': 8, 'humidity': 73, 'timestamp': '2025-03-20 15:28:28'}
{'city': 'Berlin', 'temperature': 17.2, 'humidity': 31, 'timestamp': '2025-03-20 15:28:28'}
{'city': 'Paris', 'temperature': 19.51, 'humidity': 45, 'timestamp': '2025-03-20 15:28:28'}
{'city': 'Sydney', 'temperature': 21.17, 'humidity': 82, 'timestamp': '2025-03-20 15:28:29'}
{'city': 'Los Angeles', 'temperature': 12.9, 'humidity': 59, 'timestamp': '2025-03-20 15:28:29'}
{'city': 'Madrid', 'temperature': 15.21, 'humidity': 70, 'timestamp': '2025-03-20 15:28:29'}
{'city': 'Rome', 'temperature': 9.95, 'humidity': 53, 'timestamp': '2025-03-20 15:28:29'}
{'city': 'Moscow', 'temperature': 7.59, 'humidity': 47, 'timestamp': '2025-03-20 15:28:29'}
{'city': 'Cairo', 'temperature': 16.42, 'humidity': 39, 'timestamp': '2025-0

KeyboardInterrupt: 

In [29]:
conn = sqlite3.connect("weather_data.db")
df = pd.read_sql_query("SELECT * FROM weather_log", conn)
conn.close()
print(df)

      id       city  temperature  humidity            timestamp
0      1   New York         7.37      94.0  2025-03-20 20:26:22
1      2     London        19.56      50.0  2025-03-20 20:26:23
2      3      Tokyo         8.25      71.0  2025-03-20 20:26:23
3      4     Berlin        17.43      31.0  2025-03-20 20:26:23
4      5      Paris        19.81      45.0  2025-03-20 20:26:23
..   ...        ...          ...       ...                  ...
155  156      Dubai        29.96      35.0  2025-03-20 15:31:48
156  157  Singapore        23.67      87.0  2025-03-20 15:31:55
157  158      Seoul         7.76      87.0  2025-03-20 15:31:55
158  159    Toronto        10.05      80.0  2025-03-20 15:31:55
159  160     Mumbai        29.99      39.0  2025-03-20 15:31:55

[160 rows x 5 columns]


In [32]:
calculate_aggregates()